In [30]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import xlrd
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.parsing.preprocessing import preprocess_string, strip_punctuation, strip_numeric
import gensim
from gensim import corpora
from gensim.models.coherencemodel import CoherenceModel
import nltk

In [31]:
#!pip install xlrd

In [32]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [33]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    if doc is not None:
      stop_free = " ".join([i for i in str(doc).lower().split() if i not in stop])
      punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
      normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
      return normalized
    else:
      return doc

In [34]:
file_path = ("capstone dataset.xlsx")
data = pd.read_excel(file_path, sheet_name='Workers from all websites')

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
electric_data = data['DESCRIPTION']
doc_clean = [clean(doc).split() for doc in electric_data] 
dictionary = corpora.Dictionary(doc_clean) 
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean] 

In [37]:
base_model_lda = gensim.models.ldamodel.LdaModel
ldamodel = base_model_lda(doc_term_matrix, 
                          id2word = dictionary, passes=50)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [38]:
def predict(new_text):
  if new_text is not None and str(new_text).strip() != '':
    cleaned_data = clean(new_text).split()
    bow_vector = dictionary.doc2bow(cleaned_data)
    filters = [lambda x: x.lower(), strip_punctuation, strip_numeric]
    for index, score in sorted(ldamodel[bow_vector], key=lambda tup: -1*tup[1]):
      res = preprocess_string(ldamodel.print_topic(index, 5), filters)
      return res
  else:
    return None

In [39]:
for d in electric_data:
  print(predict(d))

['electrical', 'service', 'installation', 'aspect', 'home']
['service', 'electrical', 'provide', 'job', 'quote']
['installation', 'lighting', 'electrical', 'power', 'point']
['electrical', 'service', 'home', 'system', 'also']
['electricianshandyman', 'great', 'floor', 'bathroom', 'kitchen']
['electrician', 'qualified', 'grade', '‚ì', 'based']
['electrical', 'need', 'job', 'year', 'big']
['commercial', 'enquiry', 'nbn', 'industrial', 'hire']
['electrical', 'service', 'home', 'system', 'also']
['solar', 'camera', 'auto', 'electrician', 'service']
['electrician', 'qualified', 'grade', '‚ì', 'based']
['service', 'electrical', 'provide', 'job', 'quote']
['electricianshandyman', 'great', 'floor', 'bathroom', 'kitchen']
['u', 'today', 'contact', 'information', 'concreting']
['electrician', 'qualified', 'grade', '‚ì', 'based']
['electrical', 'team', 'energy', 'experience', 'qualified']
['electrical', 'boat', 'job', 'quality', 'service']
['working', 'job', 'hard', 'get', 'service']
['electrical

In [40]:
coherence_model_lda = CoherenceModel(model=ldamodel, 
                                     texts=doc_clean, 
                                     dictionary=dictionary, 
                                     coherence="c_v")
coherence_lda = coherence_model_lda.get_coherence()
print('\nLDA Coherence Score: ', coherence_lda)


LDA Coherence Score:  0.35356872309964715


In [41]:
lda_model_multi = gensim.models.LdaMulticore(corpus=doc_term_matrix,
                                       id2word=dictionary,
                                       num_topics=2, 
                                       random_state=100,
                                       chunksize=100,
                                       alpha=0.01,
                                       eta=0.01,
                                       passes=5,
                                       per_word_topics=True)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [42]:
coherence_model_multicore = CoherenceModel(model=lda_model_multi, 
                                     texts=doc_clean, 
                                     dictionary=dictionary, 
                                     coherence="c_v")
coherence_multicore = coherence_model_multicore.get_coherence()
print('\nMulticore Coherence Score: ', coherence_multicore)


Multicore Coherence Score:  0.5069039817955701
